# Aggregating and Combining `pandas` DataFrames

In [52]:
import pandas as pd
import numpy as np
import requests as rq
from sklearn.preprocessing import OneHotEncoder
from zipfile import ZipFile

## Learning Goals

- Use GroupBy objects to organize and aggregate data
- Create pivot tables from DataFrames
- Combine DataFrames by merging and appending 

We'll work with the Austin Animal Center dataset and with data from King County's Department of Assessments (Seattle housing data).

### Austin Animal Center Data

In [53]:
data = rq.get('https://data.austintexas.gov/resource/9t4d-g238.json').text

In [54]:
animals = pd.read_json(data)

In [55]:
animals.head()

,animal_id,name,datetime,monthyear,date_of_birth,outcome_type,animal_type,sex_upon_outcome,age_upon_outcome,breed,color,outcome_subtype
0,A857223,Reeves,2022-05-13 19:00:00,2022-05-13T19:00:00.000,2021-08-13T00:00:00.000,Adoption,Other,Intact Male,8 months,Guinea Pig,Calico,NaN
1,A857222,Keanu,2022-05-13 18:57:00,2022-05-13T18:57:00.000,2021-08-13T00:00:00.000,Adoption,Other,Intact Male,8 months,Guinea Pig,White/Gray,NaN
2,A855890,Zena May,2022-05-13 18:48:00,2022-05-13T18:48:00.000,2013-04-25T00:00:00.000,Rto-Adopt,Cat,Spayed Female,9 years,Domestic Shorthair,Calico,NaN
3,A857239,Scoby,2022-05-13 18:44:00,2022-05-13T18:44:00.000,2011-05-13T00:00:00.000,Return to Owner,Dog,Intact Male,11 years,Chihuahua Shorthair,Yellow,NaN
4,A856899,*Pigeon,2022-05-13 18:18:00,2022-05-13T18:18:00.000,2022-03-09T00:00:00.000,Adoption,Dog,Spayed Female,2 months,Australian Cattle Dog Mix,Black/White,NaN


## Aggregating over DataFrames: `.groupby()`

Those of you familiar with SQL have probably used the GROUP BY command. (And if you haven't, you'll see it very soon!) Pandas has this, too.

The `.groupby()` method is especially useful for aggregate functions applied to the data grouped in particular ways.

In [56]:
animals.groupby('animal_type')

In [57]:
animals.columns

Index(['animal_id', 'name', 'datetime', 'monthyear', 'date_of_birth',
       'outcome_type', 'animal_type', 'sex_upon_outcome', 'age_upon_outcome',
       'breed', 'color', 'outcome_subtype'],
      dtype='object')

We can group by multiple columns, and also return a DataFrameGroupBy object

Notice the object type [DataFrameGroupBy](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html) object. 

### `.groups` and `.get_group()`

In [58]:
animals.groupby(['animal_type', 'outcome_type'])

In [59]:
# This retuns each group indexed by the group name: I.E. 'Bird', along with the row indices of each value
animals.groupby('animal_type').groups

{'Bird': [163, 486, 581, 811, 812], 'Cat': [2, 5, 6, 7, 8, 9, 11, 14, 15, 16, 17, 18, 19, 30, 31, 41, 48, 52, 53, 54, 62, 63, 64, 65, 68, 71, 72, 73, 74, 76, 77, 78, 79, 81, 87, 90, 91, 92, 93, 97, 98, 103, 104, 105, 107, 108, 109, 110, 111, 112, 113, 114, 116, 117, 118, 122, 125, 130, 131, 132, 133, 134, 135, 136, 137, 138, 156, 160, 161, 162, 164, 165, 166, 167, 168, 169, 170, 171, 177, 179, 180, 181, 191, 192, 193, 194, 195, 213, 215, 217, 220, 221, 222, 223, 224, 225, 226, 227, 228, 232, ...], 'Dog': [3, 4, 10, 12, 13, 21, 22, 23, 24, 25, 26, 27, 28, 29, 32, 33, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 45, 46, 47, 49, 50, 51, 55, 56, 57, 58, 59, 60, 61, 66, 67, 69, 70, 75, 80, 82, 83, 84, 85, 86, 88, 89, 94, 95, 96, 99, 102, 106, 115, 119, 120, 121, 123, 124, 126, 127, 128, 129, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 155, 157, 158, 159, 172, 173, 174, 175, 176, 178, 182, 183, 184, 185, 186, 187, 188, 189, 190, 197, 198, 199, ...], 'Other': [0, 1, 20, 100, 101, 149, 150, 1

Once we know we are working with a type of object, it opens up a suite of attributes and methods. One attribute we can look at is groups.

In [60]:
# Once we know the group indices, we can return the groups using those indices.
animals.groupby('animal_type').get_group('Dog')

,animal_id,name,datetime,monthyear,date_of_birth,outcome_type,animal_type,sex_upon_outcome,age_upon_outcome,breed,color,outcome_subtype
3,A857239,Scoby,2022-05-13 18:44:00,2022-05-13T18:44:00.000,2011-05-13T00:00:00.000,Return to Owner,Dog,Intact Male,11 years,Chihuahua Shorthair,Yellow,NaN
4,A856899,*Pigeon,2022-05-13 18:18:00,2022-05-13T18:18:00.000,2022-03-09T00:00:00.000,Adoption,Dog,Spayed Female,2 months,Australian Cattle Dog Mix,Black/White,NaN
10,A856643,*Luke,2022-05-13 16:54:00,2022-05-13T16:54:00.000,2021-12-05T00:00:00.000,Adoption,Dog,Neutered Male,5 months,Labrador Retriever Mix,White,NaN
12,A847926,Spice,2022-05-13 16:34:00,2022-05-13T16:34:00.000,2019-12-10T00:00:00.000,Adoption,Dog,Neutered Male,2 years,Labrador Retriever Mix,Tan,NaN
13,A853682,Baxtor,2022-05-13 16:32:00,2022-05-13T16:32:00.000,2022-02-28T00:00:00.000,Adoption,Dog,Neutered Male,2 months,Australian Cattle Dog Mix,White/Black,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
994,A737571,Pelusa,2022-04-14 19:11:00,2022-04-14T19:11:00.000,2016-01-11T00:00:00.000,Adoption,Dog,Spayed Female,6 years,German Shepherd/Siberian Husky,Tan/Black,NaN
995,A854306,*Spanky,2022-04-14 18:23:00,2022-04-14T18:23:00.000,2021-08-01T00:00:00.000,Adoption,Dog,Neutered Male,8 months,Pit Bull Mix,White,NaN
996,A854751,*Wiggles,2022-04-14 17:59:00,2022-04-14T17:59:00.000,2022-02-07T00:00:00.000,Transfer,Dog,Intact Male,2 months,Border Terrier/Chihuahua Shorthair,Tan/White,Partner
997,A855150,Punkin,2022-04-14 16:20:00,2022-04-14T16:20:00.000,2022-01-05T00:00:00.000,Transfer,Dog,Intact Male,3 months,Dachshund Longhair,Red,Partner


#### Multi-Indexing

In [61]:
# Same goes for multi index groupbys
animal_outcome = animals.groupby(['animal_type', 'outcome_type'])
animal_outcome.groups

{('Bird', 'Adoption'): [811, 812], ('Bird', 'Died'): [486], ('Bird', 'Euthanasia'): [163, 581], ('Cat', 'Adoption'): [11, 14, 16, 17, 18, 19, 30, 31, 41, 48, 52, 53, 54, 71, 97, 98, 103, 104, 105, 113, 114, 116, 122, 125, 130, 136, 137, 138, 156, 160, 161, 162, 164, 170, 171, 177, 179, 220, 221, 222, 223, 224, 225, 226, 227, 228, 232, 233, 236, 239, 240, 251, 256, 260, 262, 264, 265, 267, 296, 309, 311, 313, 315, 323, 324, 325, 328, 335, 336, 337, 351, 354, 358, 359, 363, 364, 366, 371, 379, 393, 398, 399, 400, 401, 402, 403, 404, 408, 410, 412, 413, 415, 460, 461, 462, 464, 466, 474, 480, 481, ...], ('Cat', 'Died'): [257, 258, 448, 773, 971], ('Cat', 'Disposal'): [299, 395, 544], ('Cat', 'Euthanasia'): [15, 213, 304, 416, 454, 684, 711, 764, 876, 909], ('Cat', 'Return to Owner'): [81, 215, 217, 234, 560, 575, 620, 685, 696, 760, 832, 883, 935, 963, 972, 973, 975], ('Cat', 'Rto-Adopt'): [2, 87, 593], ('Cat', 'Transfer'): [5, 6, 7, 8, 9, 62, 63, 64, 65, 68, 72, 73, 74, 76, 77, 78, 79, 9

In [62]:
# animal_outcome.groups is a dictionary, so we can access the group names using keys()
animal_outcome.groups.keys()

dict_keys([('Bird', 'Adoption'), ('Bird', 'Died'), ('Bird', 'Euthanasia'), ('Cat', 'Adoption'), ('Cat', 'Died'), ('Cat', 'Disposal'), ('Cat', 'Euthanasia'), ('Cat', 'Return to Owner'), ('Cat', 'Rto-Adopt'), ('Cat', 'Transfer'), ('Dog', 'Adoption'), ('Dog', 'Died'), ('Dog', 'Euthanasia'), ('Dog', 'Return to Owner'), ('Dog', 'Rto-Adopt'), ('Dog', 'Transfer'), ('Other', 'Adoption'), ('Other', 'Died'), ('Other', 'Disposal'), ('Other', 'Euthanasia'), ('Other', 'Transfer')])

In [63]:
# We can then get a specific group, such as Cats that were adopted
animal_outcome.get_group(('Cat', 'Adoption'))

,animal_id,name,datetime,monthyear,date_of_birth,outcome_type,animal_type,sex_upon_outcome,age_upon_outcome,breed,color,outcome_subtype
11,A856430,*Barney,2022-05-13 16:45:00,2022-05-13T16:45:00.000,2021-12-03T00:00:00.000,Adoption,Cat,Intact Male,5 months,Domestic Shorthair,Black,NaN
14,A856863,NaN,2022-05-13 16:30:00,2022-05-13T16:30:00.000,2022-03-09T00:00:00.000,Adoption,Cat,Spayed Female,2 months,Domestic Shorthair,Calico,NaN
16,A856866,NaN,2022-05-13 15:58:00,2022-05-13T15:58:00.000,2022-03-09T00:00:00.000,Adoption,Cat,Spayed Female,2 months,Domestic Shorthair,Calico,NaN
17,A855952,*Luigi,2022-05-13 15:02:00,2022-05-13T15:02:00.000,2022-03-15T00:00:00.000,Adoption,Cat,Neutered Male,1 month,Domestic Medium Hair,Blue,Foster
18,A855949,*Mario,2022-05-13 15:01:00,2022-05-13T15:01:00.000,2022-03-15T00:00:00.000,Adoption,Cat,Neutered Male,1 month,Domestic Medium Hair,Blue/White,Foster
...,...,...,...,...,...,...,...,...,...,...,...,...
945,A853638,A853638,2022-04-16 13:48:00,2022-04-16T13:48:00.000,2022-02-15T00:00:00.000,Adoption,Cat,Neutered Male,1 month,Domestic Medium Hair Mix,Brown Tabby/White,Foster
946,A853637,A853637,2022-04-16 13:39:00,2022-04-16T13:39:00.000,2022-02-15T00:00:00.000,Adoption,Cat,Neutered Male,1 month,Domestic Medium Hair Mix,Orange Tabby,Foster
958,A854970,A854970,2022-04-15 18:49:00,2022-04-15T18:49:00.000,2021-12-11T00:00:00.000,Adoption,Cat,Neutered Male,4 months,Domestic Shorthair Mix,Blue/White,NaN
959,A854969,A854969,2022-04-15 18:49:00,2022-04-15T18:49:00.000,2021-12-11T00:00:00.000,Adoption,Cat,Neutered Male,4 months,Domestic Shorthair Mix,White/Black,NaN


### Aggregating

Once again, as we will see in SQL, groupby objects are intended to be used with aggregation. In SQL, we will see that our queries that include GROUP BY require aggregation performed on columns.

We can use `.sum()`, `.mean()`, `.count()`, `.max()`, `.min()`, etc. Find a list of common aggregations [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html).

In [64]:
animals.groupby('animal_type').count()

,animal_id,name,datetime,monthyear,date_of_birth,outcome_type,sex_upon_outcome,age_upon_outcome,breed,color,outcome_subtype
animal_type,,,,,,,,,,,
Bird,5,0,5,5,5,5,5,5,5,5,3
Cat,396,335,396,396,396,396,396,396,396,396,244
Dog,541,493,541,541,541,541,541,541,541,541,243
Other,58,28,58,58,58,58,58,58,58,58,46


### Exercise

Use `.groupby()` to find the most recently born of each (main) animal type.

In [65]:
animals.groupby('animal_type')['date_of_birth'].max()

animal_type
Bird     2022-03-24T00:00:00.000
Cat      2022-05-08T00:00:00.000
Dog      2022-05-04T00:00:00.000
Other    2022-04-06T00:00:00.000
Name: date_of_birth, dtype: object

<details>
    <summary>Answer</summary>
    <code>animals.groupby('animal_type')['date_of_birth'].max()</code>
    </details>

## Pivoting a DataFrame

### `.pivot_table()`

Those of you familiar with Excel have probably used Pivot Tables. Pandas has a similar functionality.

Grouping by two different columns can be very helpful.

In [66]:
animals.groupby(by=['outcome_type', 'sex_upon_outcome']).agg(len)

animal_id  name  datetime  monthyear  \
outcome_type    sex_upon_outcome                                         
Adoption        Intact Female             9     9         9          9   
                Intact Male               4     4         4          4   
                Neutered Male           210   210       210        210   
                Spayed Female           195   195       195        195   
                Unknown                   2     2         2          2   
Died            Intact Female             1     1         1          1   
                Intact Male               3     3         3          3   
                Spayed Female             2     2         2          2   
                Unknown                   5     5         5          5   
Disposal        Intact Female             1     1         1          1   
                Intact Male               2     2         2          2   
                Unknown                   1     1         1          1   
Euthanasia      Intact Female             4     4         4          4   
                Intact Male               8     8         8          8   
                Neutered Male             3     3         3          3   
                Spayed Female             3     3         3          3   
                Unknown                  35    35        35         35   
Return to Owner Intact Female            27    27        27         27   
                Intact Male              38    38        38         38   
                Neutered Male            25    25        25         25   
                Spayed Female            18    18        18         18   
                Unknown                   1     1         1          1   
Rto-Adopt       Intact Male               1     1         1          1   
                Neutered Male             5     5         5          5   
                Spayed Female             8     8         8          8   
Transfer        Intact Female           128   128       128        128   
                Intact Male             119   119       119        119   
                Neutered Male            60    60        60         60   
                Spayed Female            44    44        44         44   
                Unknown                  38    38        38         38   

                                  date_of_birth  animal_type  \
outcome_type    sex_upon_outcome                               
Adoption        Intact Female                 9            9   
                Intact Male                   4            4   
                Neutered Male               210          210   
                Spayed Female               195          195   
                Unknown                       2            2   
Died            Intact Female                 1            1   
                Intact Male                   3            3   
                Spayed Female                 2            2   
                Unknown                       5            5   
Disposal        Intact Female                 1            1   
                Intact Male                   2            2   
                Unknown                       1            1   
Euthanasia      Intact Female                 4            4   
                Intact Male                   8            8   
                Neutered Male                 3            3   
                Spayed Female                 3            3   
                Unknown                      35           35   
Return to Owner Intact Female                27           27   
                Intact Male                  38           38   
                Neutered Male                25           25   
                Spayed Female                18           18   
                Unknown                       1            1   
Rto-Adopt       Intact Male                   1            1   
                Neutered Male                 5            5   
 

But it has the unsavory side effect of creating a two-level index. This can be a good time to use `.pivot_table()`.

(There is also a `.pivot()`. For the somewhat subtle differences, see [here](https://stackoverflow.com/questions/30960338/pandas-difference-between-pivot-and-pivot-table-why-is-only-pivot-table-workin).)

#### Example

In [67]:
df = pd.DataFrame({"sex": ["male", "male", "male", "male", "male",
                          "female", "female", "female", "female"],
                    "num_puppies": ["one", "one", "one", "two", "two",
                          "one", "one", "two", "two"],
                    "breed": ["terrier", "retriever", "retriever", "terrier",
                          "terrier", "retriever", "terrier", "terrier",
                          "retriever"],
                    "past_owners": [1, 2, 2, 3, 3, 4, 5, 6, 7],
                    "family_members": [2, 4, 5, 5, 6, 6, 8, 9, 9]})
df

,sex,num_puppies,breed,past_owners,family_members
0,male,one,terrier,1,2
1,male,one,retriever,2,4
2,male,one,retriever,2,5
3,male,two,terrier,3,5
4,male,two,terrier,3,6
5,female,one,retriever,4,6
6,female,one,terrier,5,8
7,female,two,terrier,6,9
8,female,two,retriever,7,9


In [68]:
# This first example aggregates values by taking the sum.

table = pd.pivot_table(df, values='past_owners', index=['sex', 'num_puppies'],
                     columns=['breed'], aggfunc=np.sum)
table

breed               retriever  terrier
sex    num_puppies                    
female one                4.0      5.0
       two                7.0      6.0
male   one                4.0      1.0
       two                NaN      6.0

#### Back to Austin animals

In [69]:
animals.pivot_table(index='outcome_type', columns='sex_upon_outcome', aggfunc=len)

age_upon_outcome                                          \
sex_upon_outcome    Intact Female Intact Male Neutered Male Spayed Female   
outcome_type                                                                
Adoption                      9.0         4.0         210.0         195.0   
Died                          1.0         3.0           NaN           2.0   
Disposal                      1.0         2.0           NaN           NaN   
Euthanasia                    4.0         8.0           3.0           3.0   
Return to Owner              27.0        38.0          25.0          18.0   
Rto-Adopt                     NaN         1.0           5.0           8.0   
Transfer                    128.0       119.0          60.0          44.0   

                             animal_id                            \
sex_upon_outcome Unknown Intact Female Intact Male Neutered Male   
outcome_type                                                       
Adoption             2.0           9.0         4.0         210.0   
Died                 5.0           1.0         3.0           NaN   
Disposal             1.0           1.0         2.0           NaN   
Euthanasia          35.0           4.0         8.0           3.0   
Return to Owner      1.0          27.0        38.0          25.0   
Rto-Adopt            NaN           NaN         1.0           5.0   
Transfer            38.0         128.0       119.0          60.0   

                                        ...          name              \
sex_upon_outcome Spayed Female Unknown  ... Intact Female Intact Male   
outcome_type                            ...                             
Adoption                 195.0     2.0  ...           9.0         4.0   
Died                       2.0     5.0  ...           1.0         3.0   
Disposal                   NaN     1.0  ...           1.0         2.0   
Euthanasia                 3.0    35.0  ...           4.0         8.0   
Return to Owner           18.0     1.0  ...          27.0        38.0   
Rto-Adopt                  8.0     NaN  ...           NaN         1.0   
Transfer                  44.0    38.0  ...         128.0       119.0   

                                                     outcome_subtype  \
sex_upon_outcome Neutered Male Spayed Female Unknown   Intact Female   
outcome_type                                                           
Adoption                 210.0         195.0     2.0             9.0   
Died                       NaN           2.0     5.0             1.0   
Disposal                   NaN           NaN     1.0             1.0   
Euthanasia                 3.0           3.0    35.0             4.0   
Return to Owner           25.0          18.0     1.0            27.0   
Rto-Adopt                  5.0           8.0     NaN             NaN   
Transfer                  60.0          44.0    38.0           128.0   

                                                                  
sex_upon_outcome Intact Male Neutered Male Spayed Female Unknown  
outcome_type                                                      
Adoption                 4.0         210.0         195.0     2.0  
Died                     3.0           NaN           2.0     5.0  
Disposal                 2.0           NaN           NaN     1.0  
Euthanasia               8.0           3.0           3.0    35.0  
Return to Owner         38.0          25.0          18.0     1.0  
Rto-Adopt                1.0           5.0           8.0     NaN  
Transfer               119.0          60.0          44.0    38.0  

[7 rows x 50 columns]

### Exercise

Use `.pivot_table()` to add up the number of my tasks by category. Hint: Use `sum()` as your aggregating function.

In [70]:
tasks = pd.DataFrame({'category': ['house', 'house', 'school', 'school'],
                      'descr': ['kitchen', 'laundry', 'git', 'Python'],
                      'priority': [2, 3, 4, 1], 'num_tasks': [2, 1, 2, 3]})

tasks

,category,descr,priority,num_tasks
0,house,kitchen,2,2
1,house,laundry,3,1
2,school,git,4,2
3,school,Python,1,3


In [71]:
tasks.pivot_table(values='num_tasks', index='category', aggfunc=sum)

,num_tasks
category,
house,3
school,5


<details>
    <summary>Answer</summary>
    <code>tasks.pivot_table(values='num_tasks', index='category', aggfunc=sum)</code>
    </details>

## Methods for Combining DataFrames: `.join()`, `.merge()`, `pd.concat()`

### `.join()`

In [72]:
toy1 = pd.DataFrame([[63, 142], [33, 47]], columns=['age', 'HP'])
toy2 = pd.DataFrame([[63, 100], [33, 200]], columns=['age', 'MP'])

toy1

,age,HP
0,63,142
1,33,47


In [73]:
toy2

,age,MP
0,63,100
1,33,200


In [74]:
# We can't just join these as they are, since we haven't specified our suffixes.

toy1.join(toy2)

ValueError: columns overlap but no suffix specified: Index(['age'], dtype='object')

In [75]:
toy1.join(toy2, lsuffix='1', rsuffix='2')

,age1,HP,age2,MP
0,63,142,63,100
1,33,47,33,200


If we don't want to keep both, we could set the overlapping column as the index in each DataFrame:

In [76]:
toy1.set_index('age').join(toy2.set_index('age'))

,HP,MP
age,,
63,142,100
33,47,200


For more on this method, check out the [doc](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html)!

### `.merge()`

Or we could use `.merge()`:

In [77]:
toy1.merge(toy2)

,age,HP,MP
0,63,142,100
1,33,47,200


In [78]:
ds_chars = pd.read_csv('data/ds_chars.csv', index_col=0)
ds_chars

,name,HP,home_state
0,greg,200,WA
1,miles,200,WA
2,alan,170,TX
3,alison,300,DC
4,rachel,200,TX


In [79]:
states = pd.read_csv('data/states.csv', index_col=0)
states

,state,nickname,capital
0,WA,evergreen,Olympia
1,TX,alamo,Austin
2,DC,district,Washington
3,OH,buckeye,Columbus
4,OR,beaver,Salem


### The `how` Parameter

This parameter in both `.join()` and `.merge()` tells the compiler what sort of join to effect. We'll cover this in detail when we discuss SQL.

In [80]:
ds_chars.merge(states,
               left_on='home_state',
               right_on='state',
               how='inner')

,name,HP,home_state,state,nickname,capital
0,greg,200,WA,WA,evergreen,Olympia
1,miles,200,WA,WA,evergreen,Olympia
2,alan,170,TX,TX,alamo,Austin
3,rachel,200,TX,TX,alamo,Austin
4,alison,300,DC,DC,district,Washington


In [81]:
ds_chars.merge(states,
               left_on='home_state',
               right_on='state',
               how='outer')

,name,HP,home_state,state,nickname,capital
0,greg,200.0,WA,WA,evergreen,Olympia
1,miles,200.0,WA,WA,evergreen,Olympia
2,alan,170.0,TX,TX,alamo,Austin
3,rachel,200.0,TX,TX,alamo,Austin
4,alison,300.0,DC,DC,district,Washington
5,NaN,NaN,NaN,OH,buckeye,Columbus
6,NaN,NaN,NaN,OR,beaver,Salem


### `pd.concat()`

This method takes a *list* of pandas objects as arguments.

In [82]:
ds_full = pd.concat([ds_chars, states])
ds_full

,name,HP,home_state,state,nickname,capital
0,greg,200.0,WA,NaN,NaN,NaN
1,miles,200.0,WA,NaN,NaN,NaN
2,alan,170.0,TX,NaN,NaN,NaN
3,alison,300.0,DC,NaN,NaN,NaN
4,rachel,200.0,TX,NaN,NaN,NaN
0,NaN,NaN,NaN,WA,evergreen,Olympia
1,NaN,NaN,NaN,TX,alamo,Austin
2,NaN,NaN,NaN,DC,district,Washington
3,NaN,NaN,NaN,OH,buckeye,Columbus
4,NaN,NaN,NaN,OR,beaver,Salem


`pd.concat()`–– and many other pandas operations –– make use of an `axis` parameter. For this particular method I need to specify whether I want to concatenate the DataFrames *row-wise* (`axis=0`) or *column-wise* (`axis=1`). The default is `axis=0`, so let's override that!

In [83]:
ds_full = pd.concat([ds_chars, states], axis=1)
ds_full

,name,HP,home_state,state,nickname,capital
0,greg,200,WA,WA,evergreen,Olympia
1,miles,200,WA,TX,alamo,Austin
2,alan,170,TX,DC,district,Washington
3,alison,300,DC,OH,buckeye,Columbus
4,rachel,200,TX,OR,beaver,Salem


## King County Assessments

As data scientists, we want to build a model to predict the sale price of a house in Seattle in 2019, based on its square footage. We know that the King County Department of Assessments has comprehensive data available on real property sales in the Seattle area. We need to prepare the data.

### First, get the data!

Go [here](https://info.kingcounty.gov/assessor/DataDownload/default.aspx) and download two files: "Real Property Sales" and "Residential Building". Then unzip them. (Or you can run the cells below if you prefer.)

In [ ]:
# %%bash
# cd data
# curl -o property_sales.zip https://aqua.kingcounty.gov/extranet/assessor/Real%20Property%20Sales.zip 

In [ ]:
# %%bash
# cd data
# curl -o res_bldg.zip https://aqua.kingcounty.gov/extranet/assessor/Residential%20Building.zip 

In [ ]:
# zf = ZipFile('data/property_sales.zip', 'r')
# zf.extractall('data')
# zf.close()

In [ ]:
# zf = ZipFile('data/res_bldg.zip', 'r')
# zf.extractall('data')
# zf.close()

In [ ]:
# You'll need to use a new encoding here. List of all encodings here:
# https://docs.python.org/3/library/codecs.html#standard-encodings

# Both of these csv files have many columns, so we'll just pre-select
# which ones we want to use.

sales_df = pd.read_csv('/Users/gdamico/Downloads/EXTR_RPSale.csv',
                       encoding='latin-1',
                       usecols=['Major', 'Minor', 'DocumentDate', 'SalePrice'])

In [ ]:
sales_df.info()

In [ ]:
bldg_df = pd.read_csv('~/Downloads/EXTR_ResBldg.csv',
                     usecols=['Major', 'Minor', 'SqFtTotLiving', 'ZipCode'])

In [ ]:
bldg_df.info()

In [ ]:
sales_data = pd.merge(sales_df, bldg_df, on=['Major', 'Minor'])

In [ ]:
sales_data.head()

In [ ]:
sales_data.info()

We can see right away that we're missing ZIP codes for many of the sales transactions.

In [ ]:
sales_data.loc[sales_data['ZipCode'].isna()].head()

### Exercise

What percentage of housing records are missing ZIP codes?

<details>
    <summary>Answer</summary>
    <code>sales_data['ZipCode'].isna().sum() / sales_data.shape[0]</code>
    </details>

Let's drop the rows with missing zip codes.

In [ ]:
sales_data = sales_data.loc[~sales_data['ZipCode'].isna(), :]

sales_data.head()

## Time Permitting: Data Cleaning with Pandas

### 1. Investigate and drop rows with invalid values in the SalePrice and SqFtTotLiving columns.

<details>
    <summary>One possible answer here</summary>
    <code>sales_data = sales_data[sales_data['SalePrice'] > 10000]</code>
    </details>

### 2. Investigate and handle non-numeric ZipCode values

Can you find a way to shorten ZIP+4 codes to the first five digits?

In [ ]:
def is_integer(x):
    try:
        _ = int(x)
    except ValueError:
        return False
    return True

sales_data.loc[sales_data['ZipCode'].apply(is_integer) == False, 'ZipCode'].head()

<details>
    <summary>One possible answer here</summary>
    <code>def five_digit_ZIP(x):
    try:
        return int(str(x)[:5])
    except:
        return x
sales_data['ZipCode'] = sales_data['ZipCode'].map(five_digit_ZIP)
sales_data = sales_data.loc[sales_data['ZipCode'].apply(is_integer) == True, :]
sales_data['ZipCode'] = sales_data['ZipCode'].map(int)</code>
    </details>

### 3. Add a column for PricePerSqFt



<details>
    <summary>Answer here</summary>
    <code>sales_data['PricePerSqFt'] = sales_data['SalePrice'] / sales_data['SqFtTotLiving']</code>
    </details>

### 4. Subset the data to 2021 sales only.

We can assume that the DocumentDate is approximately the sale date.

<details>
    <summary>Answer here</summary>
    <code>sales_data['DocumentDate'] = pd.to_datetime(sales_data['DocumentDate'])
sales_data = sales_data.loc[sales_data['DocumentDate'] > '12/31/2020']</code>
    </details>

### 5. What is the mean price per square foot for a house sold in Seattle in 2021?

<details>
    <summary>Answer here</summary>
    <code>sales_data['PricePerSqFt'].mean()</code>
    </details>

## Level Up: `pandas.set_option()`

We can adjust how `pandas` works by setting options in advance.

### Block Scientific Notation

For example, suppose we want to prevent numbers from being displayed in scientific notation.

In [ ]:
df = pd.DataFrame([[1e9, 2e9], [3e9, 4e9]])
df

Then we can use:

In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)

df

### See More Rows

Or suppose we want `pandas` to show more rows.

In [ ]:
df2 = pd.DataFrame(np.array(range(100)))
df2

In that case we can use:

In [ ]:
pd.set_option('display.max_rows', 100)

df2

For complete documentation, see [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/options.html).